[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/JARN_Ansible-Assessment/blob/master/Meeting_Notes_to_Google_Doc_Colab.ipynb)

See the project README for setup and usage details: [README.md](https://github.com/YOUR_GITHUB_USERNAME/JARN_Ansible-Assessment/blob/master/README.md)


## Meeting Notes → Google Doc (Colab)

This notebook converts the provided markdown meeting notes into a clean, properly formatted Google Doc using the Google Docs API.

### What this notebook does
- **Creates a new Google Doc**: Authenticates in Colab and creates a document in your Google Drive.
- **Applies structure**:
  - **Heading 1**: Main title (e.g., "Product Team Sync")
  - **Heading 2**: Section headers (e.g., "Attendees", "Agenda")
  - **Heading 3**: Sub-sections (Agenda sub-items)
- **Preserves lists**: Maintains nested bullet hierarchy and indentation.
- **Converts checkboxes**: Markdown `- [ ]` items become real Google Docs checkboxes.
- **Highlights mentions**: `@name` is styled distinctly (e.g., bold or colored).
- **Formats footer**: "Meeting recorded by" and "Duration" use a subtle, distinct style.

### How to use (in Colab)
1. **Run Setup/Authentication** cells first.
   - You'll be prompted to choose your Google account and grant access to `https://www.googleapis.com/auth/documents`.
2. **Provide input notes**:
   - Option A: Paste the markdown into a string variable (e.g., `MEETING_NOTES`).
   - Option B: Upload or read a `.md` file (e.g., from `examples/`).
3. **Run the conversion cell** to build the Google Doc.
4. **Open the document link** printed at the end and review formatting.

Example input options:
```python
# Option A: inline string
MEETING_NOTES = """
# Product Team Sync - May 15, 2023
...
"""

# Option B: from a .md file
with open("examples/product-team-sync.md", "r") as f:
    MEETING_NOTES = f.read()
```

### Troubleshooting
- **Auth window closed or failed**: Rerun the auth cell; ensure pop-ups are allowed.
- **Permission errors**: Make sure you complete the OAuth prompt in the current session.
- **API not enabled**: Rare in Colab, but if you see an API enablement error, enable Google Docs API in Google Cloud Console for your account/project.
- **Rate limits/intermittent errors**: Re-run the conversion cell after a short wait.

### Deliverables checklist
- **GitHub repo**: Includes this notebook and `README.md` with setup/run instructions.
- **Colab-ready**: Notebook runs cleanly from top to bottom.
- **Formatting features**: Headings, bullets, checkboxes, mentions, and footer styles applied.
- **Error handling**: Basic exceptions are caught and surfaced clearly.

### Notes
- This notebook uses ephemeral OAuth tokens in Colab; nothing sensitive is stored in the repo.
- If you tweak styles (e.g., mention color), adjust the style constants in the conversion cell.



# Author

Prepared by: Jai Adithya Ram Nayani

"What excites me most about Rilla is the chance to be part of a once-in-a-generation company in its earliest, most defining stage—where long hours, in-person collaboration, and uncompromising standards turn into outsized impact and exponential growth. I thrive in environments where intensity fuels innovation, and I’m energized by the idea of contributing to a culture that feels like the early days of Amazon or Apple, where every decision shapes the company’s trajectory. I want to help build that legacy—and who knows, I could be the next Soham Parekh."

— and yes, I wrote that line myself. 😉


# Markdown ➜ Google Doc (Colab)

This notebook converts markdown meeting notes into a well-formatted Google Doc using the Google Docs API.

- Creates a new Google Doc
- Applies Heading 1/2/3
- Preserves nested bullets
- Converts `- [ ]` checkboxes to real Google Docs checkboxes
- Styles `@mentions` (bold + color)
- Adds a distinct footer block

Follow the cells top-to-bottom.


In [1]:
# Install dependencies (Colab)
!pip -q install google-api-python-client google-auth-httplib2 google-auth-oauthlib markdown-it-py==3.0.0 beautifulsoup4==4.12.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 7.1 MB/s eta 0:00:00


In [7]:
# Authenticate with Google (Colab-friendly; supports uploaded OAuth JSON or Colab default)
from google.colab import auth, files
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import google.auth
import json, shutil

SCOPES = ['https://www.googleapis.com/auth/documents']

auth.authenticate_user()

print('Upload your OAuth client JSON (Desktop app) OR skip to use Colab account.')
uploaded = files.upload()
print(f'Uploaded files: {uploaded.keys()}')

creds = None
if uploaded:
  uploaded_name = list(uploaded.keys())[0]
  # Validate Desktop OAuth client JSON; if invalid, fall back to Colab default
  try:
    with open(uploaded_name, 'r') as f:
      data = json.load(f)
    if 'installed' in data and 'client_id' in data['installed']:
      if uploaded_name != 'credentials.json':
        shutil.move(uploaded_name, 'credentials.json')
      flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
      # Use console flow if available; otherwise fall back to Colab default
      if hasattr(flow, 'run_console'):
        creds = flow.run_console()
      else:
        creds, _ = google.auth.default(scopes=SCOPES)
    else:
      creds, _ = google.auth.default(scopes=SCOPES)
  except Exception:
    creds, _ = google.auth.default(scopes=SCOPES)
else:
  # No file uploaded → use Colab account auth
  creds, _ = google.auth.default(scopes=SCOPES)

docs = build('docs', 'v1', credentials=creds)

Upload your OAuth client JSON (Desktop app) OR skip to use Colab account.


Saving credentials.sample.json to credentials.sample (3).json
Uploaded files: dict_keys(['credentials.sample (3).json'])


In [8]:
# Load markdown (inline or from examples/original-meeting-notes.md)
from pathlib import Path

md_path = Path('/content/original-meeting-notes.md')  # Colab path after upload

print('Upload your markdown file (or skip to use embedded sample)')
from google.colab import files
uploaded_md = files.upload()

markdown_text = ''
if 'original-meeting-notes.md' in uploaded_md:
    markdown_text = Path('original-meeting-notes.md').read_text(encoding='utf-8')
else:
    # Embedded sample (subset) as fallback
    markdown_text = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
* User authentication flow
* Dashboard redesign
* Performance optimization
* Reduced load time by 40%
* Implemented caching solution

* Pending Items
* Mobile responsive fixes
* Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
* Navigation confusion
* Color contrast issues

### 3. Next Sprint Planning
* Priority Features
* Payment gateway integration
* User profile enhancement
* Analytics dashboard
* Technical Debt
* Code refactoring
* Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


Upload your markdown file (or skip to use embedded sample)


Saving product-team-sync.md to product-team-sync.md


In [ ]:
# Markdown parsing helpers
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup
import re

md = MarkdownIt('commonmark')


def markdown_to_html(md_text: str) -> str:
    return md.render(md_text)


def find_mentions(text: str):
    return [(m.start(), m.end()) for m in re.finditer(r'@\w+', text)]


In [18]:
# Docs API helpers for styles and inserts (with retries)

from typing import List, Dict, Any
import time
from googleapiclient.errors import HttpError

def new_document(title: str) -> str:
    doc = docs.documents().create(body={'title': title}).execute()
    return doc['documentId']

def safe_batch_update(document_id: str, requests: List[Dict[str, Any]], *, max_retries: int = 5, base_delay: float = 0.6):
    attempt = 0
    while True:
        try:
            if not requests:
                return None
            return docs.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
        except HttpError as e:
            status = getattr(e, 'status_code', None) or getattr(e, 'resp', {}).status if hasattr(e, 'resp') else None
            if status in (429, 500, 502, 503, 504) and attempt < max_retries:
                delay = base_delay * (2 ** attempt) * (1 + 0.1 * (attempt + 1))
                print(f"Transient error {status}. Waiting {delay:.2f}s before retry #{attempt+1}...")
                time.sleep(delay)
                attempt += 1
                continue
            raise

# Style maps
HEADING1 = {'namedStyleType': 'HEADING_1'}
HEADING2 = {'namedStyleType': 'HEADING_2'}
HEADING3 = {'namedStyleType': 'HEADING_3'}
NORMAL_TEXT = {'namedStyleType': 'NORMAL_TEXT'}

# Color objects MUST include the 'color' wrapper
COLOR_BLUE = { 'color': { 'rgbColor': { 'red': 0.10, 'green': 0.40, 'blue': 0.85 } } }
GREY       = { 'color': { 'rgbColor': { 'red': 0.38, 'green': 0.38, 'blue': 0.38 } } }

def insert_text(at: int, text: str, style: Dict[str, Any] = None):
    reqs = [{'insertText': {'location': {'index': at}, 'text': text}}]
    if style:
        reqs.append({
            'updateParagraphStyle': {
                'range': {'startIndex': at, 'endIndex': at + len(text)},
                'paragraphStyle': style,
                'fields': ','.join(style.keys())
            }
        })
    return reqs

def style_mentions(at: int, text: str):
    reqs = []
    for start, end in find_mentions(text):
        reqs.append({
            'updateTextStyle': {
                'range': {'startIndex': at + start, 'endIndex': at + end},
                'textStyle': {
                    'bold': True,
                    'foregroundColor': COLOR_BLUE  # full Color object
                },
                'fields': 'bold,foregroundColor'
            }
        })
    return reqs

def create_bullets(start: int, end: int, checkbox: bool = False):
    preset = 'BULLET_CHECKBOX' if checkbox else 'BULLET_DISC_CIRCLE_SQUARE'
    return [{
        'createParagraphBullets': {
            'range': {'startIndex': start, 'endIndex': end},
            'bulletPreset': preset
        }
    }]

def create_bullet_for_range(start: int, end: int, level: int, checkbox: bool, ordered: bool):
    preset = 'NUMBERED_DECIMAL_ALPHA_ROMAN' if ordered and not checkbox else (
        'BULLET_CHECKBOX' if checkbox else 'BULLET_DISC_CIRCLE_SQUARE'
    )
    return [{
        'createParagraphBullets': {
            'range': {'startIndex': start, 'endIndex': end},
            'bulletPreset': preset
        }
    }, {
        'updateParagraphStyle': {
            'range': {'startIndex': start, 'endIndex': end},
            'paragraphStyle': {
                'indentStart': {'magnitude': max(0, level) * 18.0, 'unit': 'PT'}
            },
            'fields': 'indentStart'
        }
    }]

In [ ]:
# Markdown ➜ Docs conversion

def build_requests_from_html(html: str) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(html, 'html.parser')
    requests: List[Dict[str, Any]] = []
    cursor = 1  # Docs content index starts at 1

    def append(par_text: str, style):
        nonlocal cursor
        text = par_text.rstrip() + "\n"
        requests.extend(insert_text(cursor, text, style))
        requests.extend(style_mentions(cursor, text))
        cursor += len(text)

    def append_list(items: List[str], checkbox: bool = False):
        nonlocal cursor
        start = cursor
        for item in items:
            text = item.rstrip() + "\n"
            requests.extend(insert_text(cursor, text, NORMAL_TEXT))
            requests.extend(style_mentions(cursor, text))
            cursor += len(text)
        requests.extend(create_bullets(start, cursor, checkbox=checkbox))

    # Headings and paragraphs
    for node in soup.body.children if soup.body else soup.children:
        if getattr(node, 'name', None) in ('h1', 'h2', 'h3', 'p'):
            style = HEADING1 if node.name == 'h1' else HEADING2 if node.name == 'h2' else HEADING3 if node.name == 'h3' else NORMAL_TEXT
            append(node.get_text(strip=True), style)
        elif getattr(node, 'name', None) in ('ul', 'ol'):
            items = [li.get_text(strip=True) for li in node.find_all('li', recursive=False)]
            is_checkbox = any('[ ]' in li.get_text() for li in node.find_all('li', recursive=False))
            # Strip markdown checkbox tokens for display
            items = [re.sub(r'^\s*[-*]\s*\[ \]\s*', '', it) for it in items]
            append_list(items, checkbox=is_checkbox)

    return requests


html = markdown_to_html(markdown_text)
print('Parsed HTML length:', len(html))

try:
    document_id = new_document('Meeting Notes (Converted)')
    reqs = build_requests_from_html(html)
    safe_batch_update(document_id, reqs)
    print('Created document:', f'https://docs.google.com/document/d/{document_id}/edit')
except HttpError as e:
    print('Docs API error:', e)
except Exception as ex:
    print('Unexpected error:', ex)


Parsed HTML length: 1757
Created document: https://docs.google.com/document/d/15LnQ1vIyhfK81s3tkI0cl2a3qKRTowZtqfwGaVllksc/edit


## Colab usage

1) Run the install cell
2) Run the auth cell and upload `credentials.json` (OAuth client)
3) Upload `original-meeting-notes.md` or use embedded sample
4) Run the conversion cell
5) Open the printed Docs URL

Required scope: `https://www.googleapis.com/auth/documents`


In [ ]:
# Improved conversion: nested bullets + footer styling (uses proper Color objects)
GREY = { 'color': { 'rgbColor': { 'red': 0.38, 'green': 0.38, 'blue': 0.38 } } }

def build_requests_from_html_nested(html: str) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(html, 'html.parser')
    requests: List[Dict[str, Any]] = []
    cursor = 1
    in_footer = False

    def append_paragraph(text: str, style: Dict[str, Any], text_style: Dict[str, Any] = None):
        nonlocal cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, style))
        requests.extend(style_mentions(cursor, line))
        if text_style:
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': cursor, 'endIndex': cursor + len(line)},
                    'textStyle': text_style,
                    'fields': ','.join(text_style.keys())
                }
            })
        cursor += len(line)

    def append_bullet(text: str, level: int, checkbox: bool):
        nonlocal cursor
        start = cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, NORMAL_TEXT))
        requests.extend(style_mentions(cursor, line))
        cursor += len(line)
        requests.append({
            'createParagraphBullets': {
                'range': {'startIndex': start, 'endIndex': start + len(line)},
                'bulletPreset': 'BULLET_CHECKBOX' if checkbox else 'BULLET_DISC_CIRCLE_SQUARE'
            }
        })
        requests.append({
            'updateParagraphStyle': {
                'range': {'startIndex': start, 'endIndex': start + len(line)},
                'paragraphStyle': {
                    'indentStart': {'magnitude': max(0, level) * 18.0, 'unit': 'PT'}
                },
                'fields': 'indentStart'
            }
        })

    def li_text_without_nested(li_tag) -> str:
        parts = []
        for child in li_tag.children:
            if getattr(child, 'name', None) in ('ul', 'ol'):
                continue
            parts.append(child.get_text(strip=True) if hasattr(child, 'get_text') else str(child).strip())
        return ' '.join([p for p in parts if p])

    def handle_list(list_tag, level: int):
        for li in list_tag.find_all('li', recursive=False):
            text = li_text_without_nested(li)
            is_checkbox = bool(re.match(r'^\s*(?:[-*])?\s*\[ \]\s*', text))
            display_text = re.sub(r'^\s*(?:[-*])?\s*\[ \]\s*', '', text).strip()
            append_bullet(display_text, level, is_checkbox)
            for sub in li.find_all(['ul', 'ol'], recursive=False):
                handle_list(sub, level + 1)

    for node in soup.body.children if soup.body else soup.children:
        name = getattr(node, 'name', None)
        if name == 'hr':
            in_footer = True
            continue
        if name in ('h1', 'h2', 'h3', 'p'):
            style = HEADING1 if name == 'h1' else HEADING2 if name == 'h2' else HEADING3 if name == 'h3' else NORMAL_TEXT
            text_style = {'foregroundColor': GREY} if in_footer and name == 'p' else None
            append_paragraph(node.get_text(strip=True), style, text_style)
        elif name in ('ul', 'ol'):
            handle_list(node, level=0)

    return requests

# Run improved flow
try:
    document_id = new_document('Meeting Notes (Converted)')
    html = markdown_to_html(markdown_text)
    reqs = build_requests_from_html_nested(html)
    safe_batch_update(document_id, reqs)
    print('Created document:', f'https://docs.google.com/document/d/{document_id}/edit')
except HttpError as e:
    print('Docs API error:', e)
except Exception as ex:
    print('Unexpected error:', ex)

Created document: https://docs.google.com/document/d/14qmiwfKygYUzct2vZM-_DGm1wTeRS_tJkLO-omjY3lI/edit


In [22]:
# Ordered list support and friendlier comments

def create_bullet_for_range(start: int, end: int, level: int, checkbox: bool, ordered: bool):
    """Turn the paragraph(s) in [start, end) into bullets with the right style.
    - ordered=True → numbered list
    - checkbox=True → checkbox bullets
    - level controls indentation visually
    """
    preset = 'NUMBERED_DECIMAL_ALPHA_ROMAN' if ordered and not checkbox else (
        'BULLET_CHECKBOX' if checkbox else 'BULLET_DISC_CIRCLE_SQUARE'
    )
    return [{
        'createParagraphBullets': {
            'range': {'startIndex': start, 'endIndex': end},
            'bulletPreset': preset
        }
    }, {
        'updateParagraphStyle': {
            'range': {'startIndex': start, 'endIndex': end},
            'paragraphStyle': {
                'indentStart': {'magnitude': max(0, level) * 18.0, 'unit': 'PT'}
            },
            'fields': 'indentStart'
        }
    }]


In [23]:
# Update improved converter to use ordered bullets + retries (with correct Color usage)

import re
from typing import List, Dict, Any
from bs4 import BeautifulSoup
from googleapiclient.errors import HttpError

def build_requests_from_html_nested_v2(html: str) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(html, 'html.parser')
    requests: List[Dict[str, Any]] = []
    cursor = 1
    in_footer = False

    def append_paragraph(text: str, style: Dict[str, Any], text_style: Dict[str, Any] = None):
        nonlocal cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, style))
        requests.extend(style_mentions(cursor, line))
        if text_style:
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': cursor, 'endIndex': cursor + len(line)},
                    'textStyle': text_style,  # full Color object goes here
                    'fields': ','.join(text_style.keys())
                }
            })
        cursor += len(line)

    def append_bullet(text: str, level: int, checkbox: bool, ordered: bool):
        nonlocal cursor
        start = cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, NORMAL_TEXT))
        requests.extend(style_mentions(cursor, line))
        cursor += len(line)
        requests.extend(create_bullet_for_range(start, start + len(line), level, checkbox, ordered))

    def li_text_without_nested(li_tag) -> str:
        parts = []
        for child in li_tag.children:
            if getattr(child, 'name', None) in ('ul', 'ol'):
                continue
            parts.append(child.get_text(strip=True) if hasattr(child, 'get_text') else str(child).strip())
        return ' '.join([p for p in parts if p])

    def handle_list(list_tag, level: int):
        ordered = (list_tag.name == 'ol')
        for li in list_tag.find_all('li', recursive=False):
            text = li_text_without_nested(li)
            is_checkbox = bool(re.match(r'^\s*(?:[-*])?\s*\[ \]\s*', text))
            display_text = re.sub(r'^\s*(?:[-*])?\s*\[ \]\s*', '', text).strip()
            append_bullet(display_text, level, is_checkbox, ordered)
            for sub in li.find_all(['ul', 'ol'], recursive=False):
                handle_list(sub, level + 1)

    for node in soup.body.children if soup.body else soup.children:
        name = getattr(node, 'name', None)
        if name == 'hr':
            in_footer = True
            continue
        if name in ('h1', 'h2', 'h3', 'p'):
            style = HEADING1 if name == 'h1' else HEADING2 if name == 'h2' else HEADING3 if name == 'h3' else NORMAL_TEXT
            text_style = {'foregroundColor': GREY} if in_footer and name == 'p' else None  # full Color object
            append_paragraph(node.get_text(strip=True), style, text_style)
        elif name in ('ul', 'ol'):
            handle_list(node, level=0)

    return requests

# Run v2 flow with retry wrapper
try:
    document_id = new_document('Meeting Notes (Converted)')
    html = markdown_to_html(markdown_text)
    reqs = build_requests_from_html_nested_v2(html)
    safe_batch_update(document_id, reqs)
    print('Created document:', f'https://docs.google.com/document/d/{document_id}/edit')
except HttpError as e:
    print('Docs API error:', e)
except Exception as ex:
    print('Unexpected error:', ex)

Created document: https://docs.google.com/document/d/1fSQiiC6_6L1ROOG8uKnpDpb0PCpiOHe7jSVBQf6bHZQ/edit
